In [4]:
import src.config
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.now().date()).floor('h')
fetch_data_from = current_date - timedelta(days=1)
fetch_data_to = current_date

In [ ]:
# simulate a call to a datawerehouse
from src.load import load_raw_data
#refactoring the fetch_batch_raw_data function to use the load_raw_data function

def fetch_batch_raw_data(from_date: datetime, to_date:datetime) ->pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # download 2 files from website
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

Timestamp('2024-11-03 00:00:00')